In [169]:
import numpy as np
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
response ="COVID-19 - PESQUISA DE ANTICORPOS IgG COVID-19 IgG"

In [160]:
pacientes = pd.read_csv(r'../Data/HC_PACIENTES_1.csv', sep='|')

df = pd.read_csv(r'../Data/HC_EXAMES_1.csv', sep='|')
df = pd.merge(df, pacientes)

df.insert(1, "EXAME", df['DE_EXAME']+" "+df['DE_ANALITO'])

out = df.pivot_table(index=['ID_aTENDIMENTO', 'IC_SEXO', 'AA_NASCIMENTO'], columns='EXAME', values='DE_RESULTADO', aggfunc='first')
out = out.reset_index(level=['IC_SEXO', 'AA_NASCIMENTO'])

In [161]:
convertedResult = pd.get_dummies(out, dummy_na=True)

In [162]:
#poem como nulo valores nao numericos
def try_float(x):
    try:
        return float(x)
    except (ValueError, TypeError):
        return np.nan
le = LabelEncoder()

# Fit and transform the column
out['IC_SEXO'] = le.fit_transform(out['IC_SEXO'])
#tirar colunas sem resposta 
out.dropna(subset=[response], inplace=True)
#0 negativo,1 positivo, removemos classe indeterminado
out = out[out[response].isin(["Não reagente", "Reagente"])]
out[response] = le.fit_transform(out[response])

# Apply to each value in the DataFrame
converted_df = out.applymap(try_float)
# colunas de texto sao eliminadas
converted_df.dropna(axis=1, how='all')

#preenche nulos da forma pedida pelo professor
for col in converted_df.columns:
    n_unique = converted_df[col].nunique(dropna=True)
    fill_value = 0.5 if n_unique == 2 else 0
    converted_df[col] = converted_df[col].fillna(fill_value)

/var/folders/bx/s_m61_b15zg60nmz229ccs0c0000gn/T/ipykernel_44120/2616678637.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  converted_df = out.applymap(try_float)


In [171]:
y = converted_df[response]
X = converted_df.drop(columns=[response])
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

foldsScore = []
foldStandartDeviation = []
foldAccuracy = []
#f score
foldF1 = []
FoldPrecision = []
#cobertura em portugues
FoldRecall = []
models = []

# treina o modelo para cada fold e guarda a pontuação
for train_index, test_index in kfold.split(X):
    clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    prediction = clf.predict(X_test)
    foldAccuracy.append(accuracy_score(y_test, prediction))
    foldF1.append(precision_score(y_test, prediction))
    FoldPrecision.append(precision_score(y_test, prediction))
    FoldRecall.append(recall_score(y_test, prediction))
    foldsScore.append(score)
    models.append(clf)

#media respostas
score = np.mean(foldsScore)
standartDeviation = np.std(foldsScore)

print(f"standard deviation: {standartDeviation:.4f}")
print(f"average score: {score:.4f}")
bestRecall = np.max(FoldRecall)
max_index = np.argmax(foldsScore)
print(f"best recall: {bestRecall:.4f}, with index {max_index}")



standard deviation: 0.0301
average score: 0.6422
best recall: 0.7910, with index 0


In [ ]:
 Arquitetura da rede neural
Nesta seção você deverá descrever arquitetura da rede neural utilizada e incluir os programas
em Python utilizados para treinar e rodar a rede. Não há qualquer problema que esses
programas sejam modificações dos programas fornecidos no exercício em sala, mas se na
descrição da arquitetura você simplesmente disser algo como “a mesma arquitetura do
exercício”, vou entender que você não tem ideia do que está sendo utilizado. Em particular,
o seu relatório deve explicitar Qual a função que está sendo otimizada.
É necessário descrever o número de entradas, o número de camadas, o tamanho de
cada uma das camadas escondidas; é necessário descrever a função utilizada para calcular
a perda (loss) e eventuais otimizações do usado no treinamento. É necessário descrever
como a rede é utilizada no treinamento e durante sua execução, quando iremos prever o
estágio de infecção de um paciente, mencionando formato das entradas, o tamanho dos
batches, e o formato da saída.
Entregar os programas Python u

In [167]:
clf.fit(X, y)


score: 0.9174
